In [69]:
import os
import pandas as pd
from typing import List
import json
from collections import defaultdict
from copy import copy

In [15]:
def flatten(t: List[List]) -> List:
    """flatten list of lists"""
    return [item for sublist in t for item in sublist]

In [26]:
###### final mapping sheet
ordered_columns = ['Original first level',
       'Original second level', 'Original third level', 'NLP Type', 'NLP first level',
       'NLP second level', 'NLP third level', 'NLP fourth level', 'Reversible']

hum_mapping_sheet = pd.read_csv(
    "tables/mapping_sheet_second_version_tmp.csv", usecols=ordered_columns
)[ordered_columns].reset_index(drop=True)
original_sheet = hum_mapping_sheet.copy()

In [27]:
original_levels_cols = [
    "Original first level",
    "Original second level",
    "Original third level",
]

mapped_levels_cols = [
    "NLP first level",
    "NLP second level",
    "NLP third level",
]

for one_level in original_levels_cols:
    hum_mapping_sheet[one_level] = hum_mapping_sheet[one_level].apply(
        lambda x: x.lower().replace("\t", "").replace("•", "").strip()
        if type(x) is str
        else x
    )

for one_level in mapped_levels_cols:
    hum_mapping_sheet[one_level] = hum_mapping_sheet[one_level].apply(
        lambda x: x.capitalize().strip() if type(x) is str else x
    )

hum_mapping_sheet.drop_duplicates(inplace=True)

hum_mapping_sheet.shape

(3157, 9)

In [28]:
nlp_mapping_sheet = hum_mapping_sheet[hum_mapping_sheet['Reversible']=='TRUE'].drop_duplicates()
nlp_mapping_sheet.shape

(637, 9)

In [35]:
nlp_mapping_sheet = nlp_mapping_sheet[nlp_mapping_sheet['Original third level'].apply(lambda x: str(x)=='nan')]

In [106]:
NLP_TYPES = [
    "2D",
    "nan",
    "1D",
    "Sector",
    "DEMOGRAPHIC GROUPS",
    "SPECIFIC NEEDS GROUPS",
    "AFFECTED GROUPS",
    "SEVERITY",
    "RELIABILITY",
]

#TODO: review this and add demographic groups new level

def get_final_nlp_name(row: pd.Series):

    final_outputs_one_row = []

    if str(row["NLP Type"]) != "nan" and str(row["NLP first level"]) != "nan":

        final_str = row["NLP Type"].lower().replace("•", "").replace("\t", "").strip().replace(" ", "_")

        ### affected groups
        if final_str == "affected_groups":
            if row["NLP first level"] == "Affected" and str(row['NLP second level'])!='nan':

                if str(row['NLP fourth level'])!='nan':
                    final_outputs_one_row.append(
                        f"secondary_tags->{row['NLP second level'].capitalize()}->{row['NLP fourth level'].capitalize()}"
                    )
                elif str(row['NLP third level'])!='nan':
                    final_outputs_one_row.append(
                        f"secondary_tags->{row['NLP second level'].capitalize()}->{row['NLP third level'].capitalize()}"
                    )
                else:
                    final_outputs_one_row.append(
                        f"first_level_tags->Affected->{row['NLP second level'].capitalize()}"
                    )
                    
        ### severity and reliability and specific needs groups
        elif final_str in ["reliability", "severity", "specific_needs_groups"]:
            final_outputs_one_row.append(
                f"secondary_tags->{final_str}->{row['NLP first level'].capitalize()}"
            )

        ### demographic groups
        elif final_str == "demographic_groups":
            if str(row["NLP first level"]) != "nan":
                if str(row["NLP fourth level"]) != "nan":
                    final_outputs_one_row.append(
                        f"Secondary Tags->Demographic Groups->{row['NLP first level'].capitalize()} - {row['NLP fourth level'].capitalize()}"
                    )
                elif str(row["NLP third level"]) != "nan":
                    final_outputs_one_row.append(
                        f"secondary_tags->Demographic Groups->{row['NLP first level'].capitalize()} - {row['NLP third level'].capitalize()}"
                    )
                elif str(row["NLP second level"]) != "nan":
                    final_outputs_one_row.append(
                        f"secondary_tags->Demographic Groups->{row['NLP first level'].capitalize()} - {row['NLP second level'].capitalize()}"
                    )                
                else:
                    final_outputs_one_row.append(
                        f"secondary_tags->Demographic Groups->{row['NLP first level'].capitalize()}"
                    )

        elif final_str == "sector":
            final_outputs_one_row.append(
                f"first_level_tags->sectors->{row['NLP first level'].capitalize()}"
            )
            if str(row["NLP second level"]) != "nan":
                final_outputs_one_row.append(
                    f"subsectors->{row['NLP first level'].capitalize()}->{row['NLP second level'].capitalize()}"
                )

        ### 1d and 2d subpillars
        elif final_str in ["1d", "2d"]:
            # sometimes only pillar
            if str(row["NLP second level"]) != "nan":
                final_outputs_one_row.append(
                    f"subpillars_{final_str[0]}d->{row['NLP first level'].capitalize()}->{row['NLP second level'].capitalize()}"
                )
            else:
                final_outputs_one_row.append(
                    f"first_level_tags->pillars_{final_str[0]}d->{row['NLP first level'].capitalize()}"
                )

        else:
            print(final_str)
            raise (Exception("problem!"))

    return final_outputs_one_row

In [107]:
nlp_mapping_sheet['mapped_nlp'] = hum_mapping_sheet.apply(
    lambda x: get_final_nlp_name(x), axis=1
)

nlp_tags_mapping_sheet = sorted(list(set(flatten(nlp_mapping_sheet['mapped_nlp']))))
len(nlp_tags_mapping_sheet)

<ipython-input-107-859a1f91943d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nlp_mapping_sheet['mapped_nlp'] = hum_mapping_sheet.apply(


166

In [108]:
nlp_tags_mapping_sheet

['Secondary Tags->Demographic Groups->All - 12-17 years old',
 'Secondary Tags->Demographic Groups->All - 3-5 years old',
 'Secondary Tags->Demographic Groups->All - 5-11 years old',
 'Secondary Tags->Demographic Groups->All - <2 years old',
 'Secondary Tags->Demographic Groups->Female - 12-17 years old',
 'Secondary Tags->Demographic Groups->Female - 5-11 years old',
 'Secondary Tags->Demographic Groups->Male - 12-17 years old',
 'Secondary Tags->Demographic Groups->Male - 5-11 years old',
 'first_level_tags->Affected->Displaced',
 'first_level_tags->pillars_1d->Casualties',
 'first_level_tags->pillars_1d->Covid-19',
 'first_level_tags->pillars_1d->Humanitarian access',
 'first_level_tags->pillars_1d->Information and communication',
 'first_level_tags->pillars_2d->Humanitarian conditions',
 'first_level_tags->pillars_2d->Priority needs',
 'first_level_tags->sectors->Agriculture',
 'first_level_tags->sectors->Cross',
 'first_level_tags->sectors->Education',
 'first_level_tags->sectors-

In [46]:
nlp_mapping_sheet

,Original first level,Original second level,Original third level,NLP Type,NLP first level,NLP second level,NLP third level,NLP fourth level,Reversible,mapped_nlp
1,casualties,dead,NaN,1D,Casualties,Dead,NaN,NaN,TRUE,[subpillars_1d->Casualties->Dead]
6,casualties,death,NaN,1D,Casualties,Dead,NaN,NaN,TRUE,[subpillars_1d->Casualties->Dead]
8,casualties,injured,NaN,1D,Casualties,Injured,NaN,NaN,TRUE,[subpillars_1d->Casualties->Injured]
14,casualties/caseloads,injured,NaN,1D,Casualties,Injured,NaN,NaN,TRUE,[subpillars_1d->Casualties->Injured]
15,casualties,missing,NaN,1D,Casualties,Missing,NaN,NaN,TRUE,[subpillars_1d->Casualties->Missing]
...,...,...,...,...,...,...,...,...,...,...
4206,población con discapacidad,NaN,NaN,SPECIFIC NEEDS GROUPS,Persons with disability,NaN,NaN,NaN,TRUE,[secondary_tags->specific_needs_groups->Person...
4207,vulnerability/pin,children with disabilities,NaN,SPECIFIC NEEDS GROUPS,Persons with disability,NaN,NaN,NaN,TRUE,[secondary_tags->specific_needs_groups->Person...
4209,gender-based group,pregnant women/lactating mothers,NaN,SPECIFIC NEEDS GROUPS,Pregnant or lactating women,NaN,NaN,NaN,TRUE,[secondary_tags->specific_needs_groups->Pregna...
4214,vulnerability/pin,pregnant,NaN,SPECIFIC NEEDS GROUPS,Pregnant or lactating women,NaN,NaN,NaN,TRUE,[secondary_tags->specific_needs_groups->Pregna...


In [32]:
nlp_mapping_folder = 'nlp_mapping_files'
if not os.path.exists(nlp_mapping_folder):
    os.mkdir(nlp_mapping_folder)

In [44]:
with open(f"{nlp_mapping_folder}/all_nlp_tags.json", "w") as fp:
   json.dump(nlp_tags_mapping_sheet, fp)

In [110]:
def _clean_tag(tag: str):
    return (
        copy(tag).lower()
        .replace("->->", "->")
        .replace("-> ", "->")
        .replace(" ->", "->")
        .replace("->none", "")
        .replace("->n/a", "")
        .replace("\t", "")
        .replace("•", "")
        .replace('.', '')
        .replace(',', '')
    )

mapping_tags_nlp2original = {}

for one_nlp_key in nlp_tags_mapping_sheet:
    last_kw = one_nlp_key.split('->')[-1].lower()
    mapping_tags_nlp2original[last_kw] = [one_nlp_key]

for i, row in nlp_mapping_sheet.iterrows():
    if str(row['Original second level'])=='nan':
        orignal_key = row['Original first level']
    else:
        orignal_key = f"{row['Original first level']}->{row['Original second level']}"

    value = row['mapped_nlp']

    mapping_tags_nlp2original[orignal_key] = value

mapping_tags_nlp2original = {_clean_tag(k): v for k, v in mapping_tags_nlp2original.items()}

with open(f"{nlp_mapping_folder}/mapping_tags_nlp2original.json", 'w') as fp:
    json.dump(mapping_tags_nlp2original, fp)

In [111]:
mapping_tags_nlp2original

{'all - 12-17 years old': ['Secondary Tags->Demographic Groups->All - 12-17 years old'],
 'all - 3-5 years old': ['Secondary Tags->Demographic Groups->All - 3-5 years old'],
 'all - 5-11 years old': ['Secondary Tags->Demographic Groups->All - 5-11 years old'],
 'all - <2 years old': ['Secondary Tags->Demographic Groups->All - <2 years old'],
 'female - 12-17 years old': ['Secondary Tags->Demographic Groups->Female - 12-17 years old'],
 'female - 5-11 years old': ['Secondary Tags->Demographic Groups->Female - 5-11 years old'],
 'male - 12-17 years old': ['Secondary Tags->Demographic Groups->Male - 12-17 years old'],
 'male - 5-11 years old': ['Secondary Tags->Demographic Groups->Male - 5-11 years old'],
 'displaced': ['first_level_tags->Affected->Displaced'],
 'casualties': ['first_level_tags->pillars_1d->Casualties'],
 'covid-19': ['first_level_tags->pillars_1d->Covid-19'],
 'humanitarian access': ['first_level_tags->pillars_1d->Humanitarian access'],
 'information and communication': 

In [125]:
#script to be  fed in DEEP

import enchant
from copy import copy
from typing import List, Dict
from collections import Counter
import json


def _clean_tag(tag: str):
    return (
        copy(tag).lower()
        .replace("->->", "->")
        .replace("-> ", "->")
        .replace(" ->", "->")
        .replace("->none", "")
        .replace("->n/a", "")
        .replace("\t", "")
        .replace("•", "")
        .replace('.', '')
        .replace(',', '')
        .replace("!", "")
        .replace("?", "")
    )


def _short_distance_matching(tag, matching_dict: Dict[str, List[str]]):
    keys = list(matching_dict.keys())
    distances = defaultdict(list)
    for k, v in matching_dict.items():
        dist = enchant.utils.levenshtein(k, tag)
        min_distance = 1 + (len(k) // 10)
        if dist <= min_distance:
            distances[k].append(-dist) #neg value so we get the min sum in the end

    if len(distances)==0:
        return []
    else:
        distances = {k: sum(v) for k, v in distances.items()}
        min_key = min(distances, key=distances.get)
        return matching_dict[min_key]


def _one_tag2nlp_match(tag, matching_dict) -> List[str]:
    
    if type(tag) is str:
        clean_tag = _clean_tag(tag)
        mapped_tag = matching_dict.get(clean_tag)
        if mapped_tag is None:
            return _short_distance_matching(clean_tag, matching_dict)
        else:
            return mapped_tag
    else:
        return []

def af2nlp_matching(tags: List[Dict[int, str]], matching_dict_path = "nlp_mapping_files/mapping_tags_nlp2original.json") -> List[List[str]]:
    with open(matching_dict_path, 'r') as f:
        matching_dict = json.load(f)

    return {one_original_af_tag['id']: _one_tag2nlp_match(one_original_af_tag['text'], matching_dict) for one_original_af_tag in tags}


In [126]:
af_tags_test = [
    {'id': 1, 'text': "vulnerability/pi->children with disabilitie"},
    {'id': 2, 'text': "!health."},
    {'id': 3, 'text': "Youth (18 to 24 years old)"},
    {'id': 4, 'text': "test"},
    {'id': 5, 'text': "CAPACITIES and response->national response capacity"},
    {'id': 6, 'text': "national response capacity"}
]

af2nlp_matching(af_tags_test)

{1: ['secondary_tags->specific_needs_groups->Persons with disability'],
 2: ['first_level_tags->sectors->Health'],
 3: ['secondary_tags->Demographic Groups->All - 18-24 years old'],
 4: [],
 5: ['subpillars_2d->Capacities & response->National response'],
 6: []}

In [ ]:
#TODO: children keywords add manual mapping
# children, push/pull factor, affected